In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string

In [79]:
training_df = pd.read_csv('training_twitter_x_y_train.csv')
test_df = pd.read_csv('test_twitter_x_test.csv')
training_df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [3]:
text = training_df['text'].values
sentiment = training_df['airline_sentiment']
print(text.shape,sentiment.shape)

(10980,) (10980,)


In [80]:
test_text = test_df['text'].values
print(test_text.shape)

(3660,)


In [82]:
documents = []
for i in text:
    documents.append(word_tokenize(i))

In [83]:
test_documents = []
for i in test_text:
    test_documents.append(word_tokenize(i))

In [5]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [7]:
lemmatizer = WordNetLemmatizer()

In [62]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [64]:
documents = [clean_review(document) for document in documents]

In [84]:
test_documents = [clean_review(document) for document in test_documents]

In [67]:
text_documents = [" ".join(document) for document in documents]

In [85]:
test_text_documents = [" ".join(document) for document in test_documents]

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(max_features=4000,max_df=0.8, min_df=0.001)
x_train_features = count_vec.fit_transform(text_documents)
x_train_features

<10980x1249 sparse matrix of type '<class 'numpy.int64'>'
	with 90487 stored elements in Compressed Sparse Row format>

In [88]:
x_test_featues = count_vec.transform(test_text_documents)
x_test_featues

<3660x1249 sparse matrix of type '<class 'numpy.int64'>'
	with 29974 stored elements in Compressed Sparse Row format>

In [89]:
y_train = sentiment

In [91]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=2000, n_jobs=-1)
clf.fit(x_train_features, y_train)

RandomForestClassifier(n_estimators=2000, n_jobs=-1)

In [92]:
clf.score(x_train_features,y_train)

0.9912568306010929

In [96]:
y_test=clf.predict(x_test_featues)
y_test.shape

(3660,)

In [98]:
np.savetxt('pred.csv',y_test,delimiter=',',fmt='%s')